In [1]:
data_path = 'data_raw_html/'
parsed_path = 'data_scrapped/'
!pip install selenium pandas  lxml

In [2]:
import codecs

def save_file(save_path:str, content:str) -> None:
    """Generic function to save files"""
    with codecs.open(save_path, 'w') as file:
        file.write(content)
        
def open_file(path: str) -> bytes:
    with codecs.open(path, 'r') as file:
        content = file.read()
    return content

## Scrape the list of the IEOs

In [3]:
from selenium import webdriver
import os

from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
list_ieos = 'https://icobench.com/ieo'
driver = webdriver.Remote('http://172.17.0.1:4444/wd/hub',DesiredCapabilities.CHROME)
driver.get(list_ieos)
path_to_save = os.path.join(data_path,'ieo_list_raw_html/raw_list.html')
print(path_to_save)
save_file(path_to_save,driver.page_source)
driver.close()

data_raw_html/ieo_list_raw_html/raw_list.html


## Parse the list of the Ieos

In [9]:
#//*[@id="ieocols"]/div/div/table
def get_links_to_proj(root):
    all_links = "//a[@class='name notranslate']/@href"
    get_all = root.xpath(all_links)
    return get_all
    

from lxml import html
import pandas as pd
path_to_open = os.path.join(data_path,'ieo_list_raw_html/raw_list.html')
content = open_file(path_to_open)
root = html.fromstring(content)
table_content = html.tostring(root.xpath('//*[@id="ieocols"]/div/div/table')[0])
df = pd.read_html(table_content)[0]
df['links'] = get_links_to_proj(root)
df = df.drop(['Review','Project'],axis=1)
df= df.rename(columns={"Project.1": "Project_Name"})
save_path = os.path.join(parsed_path,'scrapped_list.csv')
df.to_csv(save_path,index=False)
df

,Project_Name,Progress,Launchpad,Rating,Token,Start,links
0,BITFINEX,Ended$1000M,Bitfinex Launchpad,3.8,LEO,"05 May, 2019",/ico/bitfinex
1,Tron Game Global,Ended$80M,Multiple,2.6,GTP,"13 Jun, 2019",/ico/tron-game-global
2,Perlin,Ended$53M,Binance Launchpad,3.5,PERL,"24 Aug, 2019",/ico/perlin
3,Percival,Ended$35M,Coineal Launchpad,2.3,XPV,"05 Mar, 2019",/ico/percival
4,Bread,Ended$32M,Binance Launchpad,3.4,BRD,"16 Dec, 2017",/ico/bread
...,...,...,...,...,...,...,...
290,Level01,Ended—,p2pb2b Launchpad,4.3,LVX,"01 Jul, 2018",/ico/lvx
291,Ubecoin,Ended—,Latoken Launchpad,3.8,UBE,"12 Apr, 2018",/ico/ubecoin
292,INTELLIGENT HERO,Ended—,Exmarkets Launchpad,3.2,Intelligent Hero,"10 Jan, 2010",/ico/intelligent-hero
293,Rowan Energy Blockchain,22%,Latoken Launchpad,4.9,RWN,"01 Jul, 2020",/ico/rowan-energy-blockchain


## Scrape The Projects' websites

In [ ]:
import time
import urllib.parse
from selenium.webdriver.common.keys import Keys

def create_url(extended_path,base_path='https://icobench.com/'):
    url = urllib.parse.urljoin(base_path, extended_path)
    return url

def scroll_to_the_end_html(driver) -> None:
    """Webdriver is scrolling to the end of the webpage and renders the content for the
    further scrapping."""
    html = driver.find_element_by_tag_name('html')
    html.send_keys(Keys.END)

driver = webdriver.Remote('http://172.17.0.1:4444/wd/hub',DesiredCapabilities.CHROME)
df_scrape_path = os.path.join(parsed_path,'scrapped_list.csv')
df_scrape = pd.read_csv(df_scrape_path)
df_scrape['links'].apply(lambda x: create_url(x))
for i,row in df_scrape.iterrows():
    url_scrape = create_url(row['links'])
    driver.get(url_scrape)
    time.sleep(1)
    scroll_to_the_end_html(driver)
    page_source = driver.page_source
    if i % 10 == 0: 
        time.sleep(1)
    
    path_to_save = os.path.join(data_path,f'ieo_infos_raw_html/{row["Project_Name"]}_info.html')
    print(path_to_save)
    save_file(path_to_save,page_source)
    
driver.close()

data_raw_html/ieo_infos_raw_html/BITFINEX_info.html
data_raw_html/ieo_infos_raw_html/Tron Game Global_info.html
data_raw_html/ieo_infos_raw_html/Perlin_info.html
data_raw_html/ieo_infos_raw_html/Percival_info.html
data_raw_html/ieo_infos_raw_html/Bread_info.html
data_raw_html/ieo_infos_raw_html/SocialGood_info.html
data_raw_html/ieo_infos_raw_html/CharS_info.html
data_raw_html/ieo_infos_raw_html/GIFTO_info.html
data_raw_html/ieo_infos_raw_html/Windhan Energy_info.html
data_raw_html/ieo_infos_raw_html/ioeX_info.html
data_raw_html/ieo_infos_raw_html/harmony_info.html
data_raw_html/ieo_infos_raw_html/MultiVAC_info.html
data_raw_html/ieo_infos_raw_html/COTI_info.html
data_raw_html/ieo_infos_raw_html/Akropolis_info.html
data_raw_html/ieo_infos_raw_html/ECOMI_info.html
data_raw_html/ieo_infos_raw_html/Band Protocol_info.html
data_raw_html/ieo_infos_raw_html/Squeezer_info.html
data_raw_html/ieo_infos_raw_html/Azbit_info.html
data_raw_html/ieo_infos_raw_html/Menapay_info.html
data_raw_html/ieo